In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class BayesianNetwork:
    def __init__(self, structure):
        self.structure = structure
        self.nodes = {}
        self.parents = {}
        self.values_to_indices = {}

    def add_cpd(self, node, cpd, parents=None, values=None):
        self.nodes[node] = cpd
        if values:
            self.values_to_indices[node] = {value: i for i, value in enumerate(values)}
        else:
            self.values_to_indices[node] = {i: i for i in range(cpd.shape[0])}

        if parents:
            for parent in parents:
                if parent not in self.parents:
                    self.parents[parent] = []
                self.parents[parent].append(node)

    def prior_sampling(self, n_samples):
        samples = []
        for _ in range(n_samples):
            row = {}
            for node in self.nodes_order:
                if node in self.parents:
                    parent_values = tuple(int(row[parent]) for parent in self.parents[node])
                    parent_indices = tuple(self.nodes[parent]['value_to_index'][row[parent]] for parent in self.parents[node])
                    p_values = np.asarray(self.nodes[node]['cpd'][parent_indices]).flatten()
                    p_values /= p_values.sum()
                else:
                    p_values = np.asarray(self.nodes[node]['cpd']).flatten()
                    p_values /= p_values.sum()

                value = np.random.choice(self.nodes[node]['values'], p=p_values)
                row[node] = value
            samples.append(row)
        return samples


    def variable_elimination(self, target, evidence=None):
        factors = self.nodes.copy()

        # Apply evidence
        if evidence:
            for node, value in evidence.items():
                factors[node] = np.zeros_like(factors[node])
                factors[node][value] = 1

        # Eliminate non-target variables
        while len(factors) > 1:
            elimination_var = next(iter(set(factors.keys()) - {target}))
            elimination_factors = []

            for parent, children in self.parents.items():
                if elimination_var in children:
                    elimination_factors.append((parent, elimination_var))

            if elimination_factors:
                parent, child = elimination_factors.pop()
                new_cpd = np.outer(factors[parent], factors[child]).reshape((*factors[parent].shape, *factors[child].shape))
                new_cpd = new_cpd.sum(axis=1)

                for grandparent, _ in elimination_factors:
                    new_cpd = new_cpd[np.newaxis, ...].repeat(factors[grandparent].shape[0], axis=0)
                    new_cpd *= factors[grandparent][:, np.newaxis, ...]
                    new_cpd = new_cpd.sum(axis=1)

                factors[parent] = new_cpd
                self.parents[parent].remove(child)

            del factors[elimination_var]

        return factors[target]

    def is_d_separated(self, node1, node2, evidence=None):
        if not evidence:
            evidence = set()

        visited = set()
        stack = [node1]

        while stack:
            current = stack.pop()
            visited.add(current)

            if current == node2:
                return False

            for parent in (self.parents.get(current, []) + [k for k, v in self.parents.items() if current in v]):
                if parent not in visited and parent not in evidence:
                    stack.append(parent)

        return True

In [ ]:
# Define the Bayesian network structure
bn = BayesianNetwork([('Cold', 'Cough'), ('Flu', 'Cough')])

# Define the probability distributions (CPDs)
cpd_cold = np.array([0.8, 0.2])
cpd_flu = np.array([0.95, 0.05])
cpd_cough = np.array([[[0.9, 0.1], [0.6, 0.4]], [[0.7, 0.3], [0.1, 0.9]]])

# Add the probability distributions to the network
bn.add_cpd('Cold', cpd_cold, values=[0, 1])
bn.add_cpd('Flu', cpd_flu, values=[0, 1])
bn.add_cpd('Cough', cpd_cough, parents=['Cold', 'Flu'], values=[0, 1])

### Variable Elimination


In [22]:
# Perform exact inference using Variable Elimination
prob_cough = bn.variable_elimination('Cough')
print("P(Cough):", prob_cough)

prob_cough_given_cold = bn.variable_elimination('Cough', evidence={'Cold': 1})
print("P(Cough | Cold):", prob_cough_given_cold)

prob_cough_given_flu = bn.variable_elimination('Cough', evidence={'Flu': 1})
print("P(Cough | Flu):", prob_cough_given_flu)

P(Cough): [[[0.9 0.1]
  [0.6 0.4]]

 [[0.7 0.3]
  [0.1 0.9]]]
P(Cough | Cold): [[[0.9 0.1]
  [0.6 0.4]]

 [[0.7 0.3]
  [0.1 0.9]]]
P(Cough | Flu): [[[0.9 0.1]
  [0.6 0.4]]

 [[0.7 0.3]
  [0.1 0.9]]]


### Prior Sampling

In [ ]:
# Perform approximate inference using Prior Sampling
samples = bn.prior_sampling(n_samples=10000)

# Calculate the probability of having a cough
prob_cough_sampled = samples['Cough'].mean()
print("P(Cough) (sampled) ≈", prob_cough_sampled)

# Calculate the probability of having a cough given a cold
prob_cough_given_cold_sampled = samples[samples['Cold'] == 1]['Cough'].mean()
print("P(Cough | Cold) (sampled) ≈", prob_cough_given_cold_sampled)

# Calculate the probability of having a cough given the flu
prob_cough_given_flu_sampled = samples[samples['Flu'] == 1]['Cough'].mean()
print("P(Cough | Flu) (sampled) ≈", prob_cough_given_flu_sampled)

### d-separation

In [24]:
# Check d-separation between Cold and Flu given no evidence
print("Cold and Flu are d-separated (no evidence):", bn.is_d_separated('Cold', 'Flu'))

# Check d-separation between Cold and Flu given Cough
print("Cold and Flu are d-separated (Cough as evidence):", bn.is_d_separated('Cold', 'Flu', evidence={'Cough'}))

Cold and Flu are d-separated (no evidence): False
Cold and Flu are d-separated (Cough as evidence): True
